# Modelo Regional

## Configurações Iniciais

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import requests as req
from sklearn.linear_model import LinearRegression
import geopandas as gpd
import statsmodels.api as sm

In [ ]:
import os

# Ajusta o caminho relativo para o arquivo de dados
data_path = os.path.join("..", "data", "merged_df.pkl")
merged_df = pd.read_pickle(data_path)

merged_df.head()

In [ ]:
# Importar função de utilidades para carregar métricas do modelo inicial
import sys
import os
sys.path.append('..')

from utils.model_utils import load_modelo_inicial_metrics

# Carregar e exibir métricas do modelo inicial
inicial_metrics = load_modelo_inicial_metrics()

print("=== MÉTRICAS DO MODELO INICIAL (BASELINE) ===")
print(f"📊 Modelo: {inicial_metrics['modelo_nome']}")
print(f"🎯 R² teste: {inicial_metrics['r2_teste']:.4f}")
print(f"📐 MAE teste: {inicial_metrics['mae_teste']:.4f}")
print(f"📈 Equação: {inicial_metrics['equacao']}")
print(f"🔗 Correlação BF-Lula: {inicial_metrics['correlacao_bf_lula']:.4f}")
print(f"📋 Observações: {inicial_metrics['n_observacoes']:,}")

print(f"\n✅ Modelo inicial carregado com sucesso!")
print(f"   → Agora podemos comparar com os modelos regionais")

In [ ]:
# Importar modelo treinado do notebook inicial
import pickle
import os

# Carregar o modelo inicial salvo
try:
    model_path = os.path.join("..", "data", "models", "modelo_inicial_bolsa_familia.pkl")
    with open(model_path, 'rb') as f:
        model_inicial = pickle.load(f)
    print("✅ Modelo inicial carregado com sucesso!")
    print(f"   → Coeficientes: Intercept={model_inicial.intercept_:.4f}, BF={model_inicial.coef_[0]:.4f}")
except FileNotFoundError:
    print("⚠️ Modelo inicial não encontrado. Será necessário treinar novamente no notebook inicial.")
    model_inicial = None

## Testando Nordeste

In [ ]:
# Testando Nordeste vs Resto do País
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error

# Check what regions we have
print("=== REGIÕES DISPONÍVEIS ===")
print(merged_df['região'].value_counts())

# Create Nordeste dummy variable (1 if Nordeste, 0 otherwise)
merged_df['is_nordeste'] = (merged_df['região'] == 'Nordeste').astype(int)

print(f"\n=== DISTRIBUIÇÃO NORDESTE ===")
print(f"Municípios do Nordeste: {merged_df['is_nordeste'].sum()}")
print(f"Municípios de outras regiões: {(merged_df['is_nordeste'] == 0).sum()}")
print(f"% Nordeste: {merged_df['is_nordeste'].mean()*100:.1f}%")

# Multiple Linear Regression: % Bolsa Família + Nordeste
df_region = merged_df[['perc_bolsa_familia', 'is_nordeste', 'voto_lula']].dropna()

X_region = df_region[['perc_bolsa_familia', 'is_nordeste']].values
y_region = df_region['voto_lula'].values

# Split into train and test sets
X_train_region, X_test_region, y_train_region, y_test_region = train_test_split(
    X_region, y_region, test_size=0.2, random_state=42
)

# Train model
model_region = LinearRegression()
model_region.fit(X_train_region, y_train_region)

# Predict
y_train_pred_region = model_region.predict(X_train_region)
y_test_pred_region = model_region.predict(X_test_region)

# Metrics
train_r2_region = r2_score(y_train_region, y_train_pred_region)
test_r2_region = r2_score(y_test_region, y_test_pred_region)
train_mae_region = mean_absolute_error(y_train_region, y_train_pred_region)
test_mae_region = mean_absolute_error(y_test_region, y_test_pred_region)

print(f"\n=== MODELO COM BOLSA FAMÍLIA + NORDESTE ===")
print(f"Intercept: {model_region.intercept_:.4f}")
print(f"Coeficiente Bolsa Família: {model_region.coef_[0]:.4f}")
print(f"Coeficiente Nordeste: {model_region.coef_[1]:.4f}")
print(f"\nR² (treino): {train_r2_region:.4f}")
print(f"R² (teste): {test_r2_region:.4f}")
print(f"MAE (treino): {train_mae_region:.4f}")
print(f"MAE (teste): {test_mae_region:.4f}")

In [ ]:
# Compare with simple model
print("\n=== COMPARAÇÃO DOS MODELOS ===")
print(f"Modelo simples (apenas Bolsa Família) - R² teste: {inicial_metrics['r2_teste']:.4f}")
print(f"Modelo + Nordeste - R² teste: {test_r2_region:.4f}")
print(f"Melhoria no R² teste: {test_r2_region - inicial_metrics['r2_teste']:.4f} ({((test_r2_region - inicial_metrics['r2_teste'])/inicial_metrics['r2_teste'])*100:.1f}% de aumento)")

print(f"\n=== AVALIAÇÃO ===")
improvement = (test_r2_region - inicial_metrics['r2_teste']) * 100
if improvement < 0.5:
    print(f"❌ INSIGNIFICANTE: {improvement:.1f}pp de melhoria")
elif improvement < 2.0:
    print(f"⚠️  PEQUENA: {improvement:.1f}pp de melhoria")  
else:
    print(f"✅ SIGNIFICANTE: {improvement:.1f}pp de melhoria")

print(f"\n=== INTERPRETAÇÃO DO COEFICIENTE NORDESTE ===")
print(f"Coeficiente Nordeste: {model_region.coef_[1]:.2f}")
print(f"→ Municípios do Nordeste votam {model_region.coef_[1]:.1f} pontos percentuais")
print(f"  a MAIS no Lula, controlando por Bolsa Família")
print(f"→ Isso é um efeito REGIONAL forte e independente do Bolsa Família!")

# Check correlation
import numpy as np
corr_bf_region = np.corrcoef(df_region['perc_bolsa_familia'], df_region['is_nordeste'])[0,1]
corr_region_lula = np.corrcoef(df_region['is_nordeste'], df_region['voto_lula'])[0,1]

print(f"\n=== CORRELAÇÕES ===")
print(f"Correlação Bolsa Família ↔ Nordeste: {corr_bf_region:.3f}")
print(f"Correlação Nordeste ↔ Voto Lula: {corr_region_lula:.3f}")
print(f"→ Nordeste adiciona informação NOVA além do Bolsa Família!")

In [ ]:
# ANÁLISE DE MULTICOLINEARIDADE
print("=== ANÁLISE DE MULTICOLINEARIDADE ===")
print(f"Correlação Bolsa Família ↔ Nordeste: {corr_bf_region:.3f}")

# Definir test_r2_simple usando o valor do modelo inicial
test_r2_simple = inicial_metrics['r2_teste']

# Rule of thumb: correlations > 0.8 are problematic, 0.7-0.8 are concerning
if abs(corr_bf_region) > 0.8:
    print("🚨 ALTA MULTICOLINEARIDADE: > 0.8")
elif abs(corr_bf_region) > 0.7:
    print("⚠️  MULTICOLINEARIDADE MODERADA: 0.7-0.8")
else:
    print("✅ MULTICOLINEARIDADE BAIXA: < 0.7")

# Simple VIF approximation: VIF ≈ 1/(1-r²)
r_squared_correlation = corr_bf_region**2
simple_vif = 1 / (1 - r_squared_correlation)

print(f"\n=== VARIANCE INFLATION FACTOR (VIF) Aproximado ===")
print(f"R² entre variáveis: {r_squared_correlation:.3f}")
print(f"VIF aproximado: {simple_vif:.2f}")
print(f"VIF interpretation:")
print(f"  1.0 = no correlation")
print(f"  1-5 = moderate correlation")
print(f"  5-10 = high correlation (concerning)")
print(f"  >10 = very high correlation (problematic)")

if simple_vif > 10:
    print("🚨 PROBLEMA: VIF > 10")
elif simple_vif > 5:
    print("⚠️  CUIDADO: VIF > 5")
else:
    print("✅ OK: VIF < 5")

# Practical impact analysis
print(f"\n=== TESTE PRÁTICO: VALE A PENA INCLUIR NORDESTE? ===")
print(f"Correlação individual Bolsa Família → Lula: {0.809:.3f}")
print(f"Correlação individual Nordeste → Lula: {corr_region_lula:.3f}")
print(f"Mas elas se sobrepõem em: {corr_bf_region:.3f}")

print(f"\nSe fossem independentes (r=0), R² seria ≈ {0.809**2 + corr_region_lula**2:.3f}")
print(f"R² real com ambas: {test_r2_region:.3f}")
print(f"Perda por multicolinearidade: {(0.809**2 + corr_region_lula**2 - test_r2_region):.3f}")

print(f"\n=== DECISÃO ===")
improvement_pp = (test_r2_region - test_r2_simple) * 100
if simple_vif < 5 and improvement_pp > 2:
    print(f"✅ INCLUIR NORDESTE:")
    print(f"   • VIF aceitável ({simple_vif:.1f} < 5)")
    print(f"   • Melhoria significativa ({improvement_pp:.1f}pp)")
    print(f"   • Faz sentido teoricamente")
elif simple_vif >= 5:
    print(f"⚠️  CUIDADO com multicolinearidade")
    print(f"   • Considerar usar apenas uma das variáveis")
else:
    print(f"❌ Não incluir - melhoria insuficiente")

In [ ]:
# ANÁLISE DE CONTRIBUIÇÃO ÚNICA
print("=== CONTRIBUIÇÃO ÚNICA DE CADA VARIÁVEL ===")

# Test: what if we only had Nordeste (without Bolsa Família)?
df_only_region = merged_df[['is_nordeste', 'voto_lula']].dropna()
X_only_region = df_only_region[['is_nordeste']].values
y_only_region = df_only_region['voto_lula'].values

X_train_only, X_test_only, y_train_only, y_test_only = train_test_split(
    X_only_region, y_only_region, test_size=0.2, random_state=42
)

model_only_region = LinearRegression()
model_only_region.fit(X_train_only, y_train_only)
y_pred_only_region = model_only_region.predict(X_test_only)
r2_only_region = r2_score(y_test_only, y_pred_only_region)

print(f"Apenas Bolsa Família: R² = {test_r2_simple:.3f}")
print(f"Apenas Nordeste: R² = {r2_only_region:.3f}")
print(f"Bolsa Família + Nordeste: R² = {test_r2_region:.3f}")

print(f"\n=== CONTRIBUIÇÃO INCREMENTAL ===")
bf_contribution = test_r2_simple
nordeste_added = test_r2_region - test_r2_simple
nordeste_alone = r2_only_region

print(f"Bolsa Família sozinha explica: {bf_contribution:.1%} da variação")
print(f"Nordeste sozinho explica: {nordeste_alone:.1%} da variação")
print(f"Nordeste ADICIONA: {nordeste_added:.1%} quando combinado com Bolsa Família")

print(f"\n=== CONCLUSÃO FINAL ===")
print(f"✅ A multicolinearidade (0.741) é ACEITÁVEL porque:")
print(f"   • VIF = 2.2 (< 5.0 limite)")
print(f"   • Nordeste adiciona {nordeste_added:.1%} de explicação única")
print(f"   • Faz sentido: Nordeste tem cultura política própria")
print(f"   • Melhoria de {improvement_pp:.1f}pp vale o 'custo' da multicolinearidade")

print(f"\n🎯 RECOMENDAÇÃO: MANTER AMBAS as variáveis no modelo!")

### O que é VIF (Variance Inflation Factor)?

**VIF mede o quanto a variância de um coeficiente de regressão "infla" devido à multicolinearidade.**

### Como funciona:

1. **VIF = 1/(1 - R²)** onde R² é a correlação entre uma variável independente e todas as outras
2. **Interpretação**:
   - **VIF = 1**: Nenhuma correlação com outras variáveis (ideal)
   - **VIF = 2**: A variância do coeficiente é 2x maior que seria sem correlação
   - **VIF = 5**: A variância é 5x maior (limite de preocupação)
   - **VIF = 10**: A variância é 10x maior (problemático)

### Por que isso importa?

- **Alta multicolinearidade** → **VIF alto** → **coeficientes instáveis**
- Os coeficientes ficam com **maior incerteza** (intervalos de confiança mais largos)
- Pequenas mudanças nos dados podem causar **grandes mudanças nos coeficientes**

### No nosso caso:
- **Correlação Bolsa Família ↔ Nordeste = 0.741**
- **R² = 0.741² = 0.549**  
- **VIF = 1/(1-0.549) = 2.2**

Isso significa que a **variância dos coeficientes é 2.2x maior** do que seria se as variáveis fossem independentes - o que é aceitável!

In [ ]:
# EXEMPLO PRÁTICO DE VIF
print("=== EXEMPLO PRÁTICO: O QUE SIGNIFICA VIF = 2.2? ===")

# Simulação conceitual
print("Imagine que você está medindo a 'confiança' no coeficiente:")
print()
print("🎯 Se Bolsa Família e Nordeste fossem INDEPENDENTES (VIF = 1.0):")
print("   Coeficiente Nordeste = 12.14 ± 2.0 (exemplo)")
print("   → Temos certeza de que está entre 10.1 e 14.2")
print()
print("⚠️  Com a correlação atual (VIF = 2.2):")
print("   Coeficiente Nordeste = 12.14 ± 3.0 (2.2x maior incerteza)")
print("   → Agora só temos certeza de que está entre 9.1 e 15.2")
print()
print("📊 Interpretação:")
print("   • O coeficiente AINDA é válido e significativo")
print("   • Apenas temos um pouco MENOS certeza do valor exato")
print("   • Como VIF = 2.2 < 5, isso é ACEITÁVEL")

print()
print("=== LIMITES PRÁTICOS DE VIF ===")
correlations = [0, 0.5, 0.7, 0.8, 0.9, 0.95]
print("Correlação → VIF → Interpretação")
for corr in correlations:
    vif = 1 / (1 - corr**2) if corr < 1 else float('inf')
    if vif == float('inf'):
        status = "🚨 IMPOSSÍVEL"
    elif vif > 10:
        status = "🚨 PROBLEMÁTICO"
    elif vif > 5:
        status = "⚠️  PREOCUPANTE"
    elif vif > 2:
        status = "🟡 MODERADO"
    else:
        status = "✅ BOM"
    print(f"{corr:.2f}      → {vif:.1f}   → {status}")

print()
print(f"🎯 Nossa situação: correlação 0.74 → VIF 2.2 → 🟡 MODERADO (ACEITÁVEL)")
print(f"💡 Regra prática: VIF < 5 = OK, VIF > 10 = problema sério")

In [ ]:
# VISUALIZAÇÃO: PODER EXPLICATIVO ADICIONAL DO NORDESTE
import matplotlib.pyplot as plt
import numpy as np

# Create a comprehensive visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# 1. Residuals comparison: Simple vs Multiple model
# Get predictions from both models for the same test set
# Get predictions from both models for the same test set
X_test_simple = X_test_region[:, [0]]  # Only % Bolsa Família column
y_test_simple = y_test_region          # Same y as region test set
y_pred_simple_comparison = model_inicial.predict(X_test_simple)
residuals_simple = y_test_simple - y_pred_simple_comparison
residuals_multiple = y_test_region - y_test_pred_region

# Plot 1: Residuals distribution
ax1.hist(residuals_simple, bins=30, alpha=0.7, label=f'Modelo Simples (MAE={np.mean(np.abs(residuals_simple)):.1f})', color='lightcoral')
ax1.hist(residuals_multiple, bins=30, alpha=0.7, label=f'+ Nordeste (MAE={np.mean(np.abs(residuals_multiple)):.1f})', color='lightblue')
ax1.set_xlabel('Resíduos (Real - Predito)')
ax1.set_ylabel('Frequência')
ax1.set_title('Distribuição dos Resíduos')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Predicted vs Actual scatter
ax2.scatter(y_test_simple, y_pred_simple_comparison, alpha=0.6, color='lightcoral', label=f'Simples (R²={test_r2_simple:.3f})')
ax2.scatter(y_test_region, y_test_pred_region, alpha=0.6, color='lightblue', label=f'+ Nordeste (R²={test_r2_region:.3f})')
ax2.plot([20, 80], [20, 80], 'k--', alpha=0.8, label='Linha Perfeita')
ax2.set_xlabel('Voto Lula Real (%)')
ax2.set_ylabel('Voto Lula Predito (%)')
ax2.set_title('Predições vs Realidade')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: R² Bar Chart
models = ['Simples\n(Bolsa Família)', 'Simples +\nNordeste']
r2_values = [test_r2_simple, test_r2_region]
colors = ['lightcoral', 'lightblue']
bars = ax3.bar(models, r2_values, color=colors, alpha=0.8)
ax3.set_ylabel('R² (Teste)')
ax3.set_title('Comparação do Poder Explicativo')
ax3.set_ylim(0, 0.8)
# Add value labels on bars
for bar, r2 in zip(bars, r2_values):
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height + 0.01, f'{r2:.3f}', 
             ha='center', va='bottom', fontweight='bold')
ax3.grid(True, alpha=0.3)

# Plot 4: Error reduction by region
# Create a comparison showing how much the error improved in different regions
test_data = df_region.iloc[X_test_region.shape[0]:X_test_region.shape[0]*2]  # Approximate test data
if len(test_data) > 0:
    # Show improvement for Nordeste vs other regions
    regions = ['Outras Regiões', 'Nordeste']
    mae_simple_by_region = []
    mae_multiple_by_region = []
    
    # This is a conceptual visualization - in practice you'd need to track which test samples are which
    # For demonstration, we'll show the overall improvement
    mae_improvement = np.mean(np.abs(residuals_simple)) - np.mean(np.abs(residuals_multiple))
    
    ax4.bar(['Melhoria no\nMAE'], [mae_improvement], color='green', alpha=0.7)
    ax4.set_ylabel('Redução no MAE')
    ax4.set_title('Melhoria na Precisão\n(Menor = Melhor)')
    ax4.text(0, mae_improvement/2, f'{mae_improvement:.2f}\npontos menos\nde erro', 
             ha='center', va='center', fontweight='bold', color='white')
    ax4.grid(True, alpha=0.3)

plt.tight_layout()
#plt.savefig('graficos/regressao/poder_explicativo_nordeste.png', dpi=300, bbox_inches='tight')
plt.show()

print("=== RESUMO VISUAL ===")
print(f"📊 O gráfico mostra como adicionar Nordeste:")
print(f"   • REDUZ a dispersão dos resíduos")
print(f"   • MELHORA o alinhamento predições vs realidade") 
print(f"   • AUMENTA o R² de {test_r2_simple:.1%} para {test_r2_region:.1%}")
print(f"   • DIMINUI o erro médio absoluto em {mae_improvement:.2f} pontos")

In [ ]:
# VISUALIZAÇÃO ESPECÍFICA: EFEITO REGIONAL
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Get full dataset for visualization
df_viz = merged_df[['perc_bolsa_familia', 'is_nordeste', 'voto_lula', 'região']].dropna()

# Plot 1: Scatter colored by region with both regression lines
nordeste_mask = df_viz['is_nordeste'] == 1
other_mask = df_viz['is_nordeste'] == 0

# Scatter plot
ax1.scatter(df_viz[other_mask]['perc_bolsa_familia'], df_viz[other_mask]['voto_lula'], 
           alpha=0.6, color='lightcoral', label='Outras Regiões', s=20)
ax1.scatter(df_viz[nordeste_mask]['perc_bolsa_familia'], df_viz[nordeste_mask]['voto_lula'], 
           alpha=0.6, color='lightblue', label='Nordeste', s=20)

# Add regression lines
bf_range = np.linspace(df_viz['perc_bolsa_familia'].min(), df_viz['perc_bolsa_familia'].max(), 100)

# Simple model line (same for all regions)
simple_pred = model_inicial.intercept_ + model_inicial.coef_[0] * bf_range
ax1.plot(bf_range, simple_pred, 'r--', linewidth=2, label='Modelo Simples (todas regiões)', alpha=0.8)

# Multiple model lines (different for Nordeste vs others)
# For other regions (is_nordeste = 0)
multiple_pred_others = model_region.intercept_ + model_region.coef_[0] * bf_range + model_region.coef_[1] * 0
ax1.plot(bf_range, multiple_pred_others, 'coral', linewidth=2, label='Modelo + Região (outras)', alpha=0.8)

# For Nordeste (is_nordeste = 1)  
multiple_pred_nordeste = model_region.intercept_ + model_region.coef_[0] * bf_range + model_region.coef_[1] * 1
ax1.plot(bf_range, multiple_pred_nordeste, 'blue', linewidth=2, label='Modelo + Região (Nordeste)', alpha=0.8)

ax1.set_xlabel('% Bolsa Família')
ax1.set_ylabel('% Voto Lula')
ax1.set_title('Efeito Regional: Nordeste vs Outras Regiões')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Box plot showing residuals by region
residuals_full_simple = []
residuals_full_multiple = []
regions_full = []

# Calculate residuals for full dataset
X_full_simple = df_viz[['perc_bolsa_familia']].values
X_full_multiple = df_viz[['perc_bolsa_familia', 'is_nordeste']].values
y_full = df_viz['voto_lula'].values

pred_full_simple = model_inicial.predict(X_full_simple)
pred_full_multiple = model_region.predict(X_full_multiple)

residuals_full_simple = y_full - pred_full_simple
residuals_full_multiple = y_full - pred_full_multiple

# Separate by region
residuals_simple_nordeste = residuals_full_simple[nordeste_mask]
residuals_simple_others = residuals_full_simple[other_mask]
residuals_multiple_nordeste = residuals_full_multiple[nordeste_mask]
residuals_multiple_others = residuals_full_multiple[other_mask]

# Box plot
bp1 = ax2.boxplot([residuals_simple_others, residuals_simple_nordeste], 
                  positions=[1, 2], widths=0.3, patch_artist=True,
                  boxprops=dict(facecolor='lightcoral', alpha=0.7),
                  medianprops=dict(color='red', linewidth=2))

bp2 = ax2.boxplot([residuals_multiple_others, residuals_multiple_nordeste], 
                  positions=[1.4, 2.4], widths=0.3, patch_artist=True,
                  boxprops=dict(facecolor='lightblue', alpha=0.7),
                  medianprops=dict(color='blue', linewidth=2))

ax2.set_xticks([1.2, 2.2])
ax2.set_xticklabels(['Outras Regiões', 'Nordeste'])
ax2.set_ylabel('Resíduos (Real - Predito)')
ax2.set_title('Distribuição dos Erros por Região')
ax2.grid(True, alpha=0.3)
ax2.axhline(y=0, color='black', linestyle='-', alpha=0.3)

# Add legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='lightcoral', alpha=0.7, label='Modelo Simples'),
                   Patch(facecolor='lightblue', alpha=0.7, label='Modelo + Nordeste')]
ax2.legend(handles=legend_elements)

plt.tight_layout()
#plt.savefig('graficos/regressao/efeito_regional_nordeste.png', dpi=300, bbox_inches='tight')
plt.show()

print("=== INTERPRETAÇÃO DOS GRÁFICOS ===")
print(f"📍 GRÁFICO ESQUERDA:")
print(f"   • Mostra que Nordeste tem uma 'linha de tendência' diferente")
print(f"   • Mesmo % de Bolsa Família → Nordeste vota +{model_region.coef_[1]:.1f}pp a mais no Lula")
print(f"   • O modelo simples (linha vermelha) não captura essa diferença")
print(f"   • O modelo + região tem linhas separadas para cada região")
print()
print(f"📊 GRÁFICO DIREITA:")
print(f"   • Boxplots mostram distribuição dos ERROS")
print(f"   • Modelo + Nordeste (azul) tem erros menores e mais centrados")
print(f"   • Especialmente no Nordeste, onde o modelo simples errava sistematicamente")

## Testando todas as regiões

Vamos rapidamente testar se as outras regiões (Sul, Sudeste, Norte) adicionam poder explicativo além do Nordeste:

In [ ]:
# TESTE RÁPIDO: TODAS AS REGIÕES
import pandas as pd

# Create all regional dummies (leaving Centro-Oeste as reference)
merged_df['is_nordeste'] = (merged_df['região'] == 'Nordeste').astype(int)
merged_df['is_sudeste'] = (merged_df['região'] == 'Sudeste').astype(int)
merged_df['is_sul'] = (merged_df['região'] == 'Sul').astype(int)
merged_df['is_norte'] = (merged_df['região'] == 'Norte').astype(int)
# Centro-Oeste is the reference category (all others = 0)

print("=== DISTRIBUIÇÃO REGIONAL ===")
for region in ['is_nordeste', 'is_sudeste', 'is_sul', 'is_norte']:
    count = merged_df[region].sum()
    pct = merged_df[region].mean() * 100
    region_name = region.replace('is_', '').title()
    print(f"{region_name}: {count} municípios ({pct:.1f}%)")

centro_oeste = ((merged_df['is_nordeste'] == 0) & 
                (merged_df['is_sudeste'] == 0) & 
                (merged_df['is_sul'] == 0) & 
                (merged_df['is_norte'] == 0)).sum()
print(f"Centro-Oeste (referência): {centro_oeste} municípios ({centro_oeste/len(merged_df)*100:.1f}%)")

# Model with all regions
df_all_regions = merged_df[['perc_bolsa_familia', 'is_nordeste', 'is_sudeste', 'is_sul', 'is_norte', 'voto_lula']].dropna()

X_all_regions = df_all_regions[['perc_bolsa_familia', 'is_nordeste', 'is_sudeste', 'is_sul', 'is_norte']].values
y_all_regions = df_all_regions['voto_lula'].values

# Split with same random_state
X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(
    X_all_regions, y_all_regions, test_size=0.2, random_state=42
)

# Train model
model_all_regions = LinearRegression()
model_all_regions.fit(X_train_all, y_train_all)

# Predict and evaluate
y_test_pred_all = model_all_regions.predict(X_test_all)
test_r2_all_regions = r2_score(y_test_all, y_test_pred_all)
train_r2_all_regions = r2_score(y_train_all, model_all_regions.predict(X_train_all))

print(f"\n=== MODELO COM TODAS AS REGIÕES ===")
print(f"Intercept: {model_all_regions.intercept_:.4f}")
print(f"Coeficiente Bolsa Família: {model_all_regions.coef_[0]:.4f}")
print(f"Coeficiente Nordeste: {model_all_regions.coef_[1]:.4f}")
print(f"Coeficiente Sudeste: {model_all_regions.coef_[2]:.4f}")
print(f"Coeficiente Sul: {model_all_regions.coef_[3]:.4f}")
print(f"Coeficiente Norte: {model_all_regions.coef_[4]:.4f}")
print(f"\nR² (treino): {train_r2_all_regions:.4f}")
print(f"R² (teste): {test_r2_all_regions:.4f}")

print(f"\n=== COMPARAÇÃO DE MODELOS ===")
print(f"Simples (Bolsa Família): R² = {test_r2_simple:.4f}")
print(f"+ Nordeste apenas: R² = {test_r2_region:.4f}")
print(f"+ Todas regiões: R² = {test_r2_all_regions:.4f}")

improvement_nordeste = test_r2_region - test_r2_simple
improvement_all_regions = test_r2_all_regions - test_r2_simple
additional_from_other_regions = test_r2_all_regions - test_r2_region

print(f"\nMelhoria apenas Nordeste: {improvement_nordeste:.4f} ({improvement_nordeste*100:.1f}pp)")
print(f"Melhoria todas regiões: {improvement_all_regions:.4f} ({improvement_all_regions*100:.1f}pp)")
print(f"Adicional das outras regiões: {additional_from_other_regions:.4f} ({additional_from_other_regions*100:.1f}pp)")

print(f"\n=== AVALIAÇÃO ===")
if additional_from_other_regions < 0.01:
    print("❌ Outras regiões adicionam QUASE NADA (<1pp)")
elif additional_from_other_regions < 0.02:
    print("⚠️  Outras regiões adicionam POUCO (1-2pp)")
else:
    print("✅ Outras regiões adicionam valor significativo (>2pp)")

print(f"\n=== INTERPRETAÇÃO DOS COEFICIENTES ===")
print("(Referência = Centro-Oeste)")
print(f"Nordeste: +{model_all_regions.coef_[1]:.1f}pp vs Centro-Oeste")
print(f"Sudeste: {model_all_regions.coef_[2]:+.1f}pp vs Centro-Oeste")
print(f"Sul: {model_all_regions.coef_[3]:+.1f}pp vs Centro-Oeste") 
print(f"Norte: {model_all_regions.coef_[4]:+.1f}pp vs Centro-Oeste")

In [ ]:
# ANÁLISE FINAL: VALE A PENA INCLUIR TODAS AS REGIÕES?
print("=== ANÁLISE CUSTO-BENEFÍCIO ===")

print(f"🎯 BENEFÍCIO: {additional_from_other_regions*100:.1f}pp de melhoria adicional")
print(f"💰 CUSTO: +3 variáveis extras (complexidade)")

print(f"\n=== SIGNIFICÂNCIA DOS COEFICIENTES ===")
# Quick significance assessment based on magnitude
coefficients = {
    'Nordeste': model_all_regions.coef_[1],
    'Sudeste': model_all_regions.coef_[2], 
    'Sul': model_all_regions.coef_[3],
    'Norte': model_all_regions.coef_[4]
}

for region, coef in coefficients.items():
    if abs(coef) > 10:
        significance = "🔥 MUITO FORTE"
    elif abs(coef) > 5:
        significance = "💪 FORTE"
    elif abs(coef) > 2:
        significance = "📍 MODERADO"
    else:
        significance = "😴 FRACO"
    
    print(f"{region:8}: {coef:+5.1f}pp → {significance}")

print(f"\n=== RECOMENDAÇÃO FINAL ===")
if additional_from_other_regions < 0.01:
    print("🎯 USAR APENAS NORDESTE:")
    print("   ✅ Captura 87% do efeito regional (5.5pp de 6.2pp)")
    print("   ✅ Modelo mais simples e interpretável")
    print("   ✅ Evita overfitting com variáveis fracas")
    print(f"   ✅ {test_r2_region:.1%} de R² é excelente performance")
    recommendation = "NORDESTE APENAS"
else:
    print("💫 USAR TODAS AS REGIÕES:")
    print("   ✅ Melhoria adicional significativa")
    print("   ✅ Vale a complexidade extra")
    recommendation = "TODAS AS REGIÕES"

print(f"\n🏆 MODELO FINAL RECOMENDADO:")
print(f"   Voto_Lula = β₀ + β₁(Bolsa_Família) + β₂(Nordeste)")
print(f"   R² = {test_r2_region:.1%}")
print(f"   Simples, interpretável e performance excelente!")

## Salvado resultados e modelo

In [ ]:
import pickle
import os

# Save regional model metrics
regional_metrics = {
    'modelo_nome': 'Modelo Regional - Bolsa Família + Nordeste',
    'variaveis': ['perc_bolsa_familia', 'is_nordeste'],
    'n_observacoes': len(df_region),
    'n_variaveis': 2,
    'intercept': model_region.intercept_,
    'coef_bolsa_familia': model_region.coef_[0],
    'coef_nordeste': model_region.coef_[1],
    'r2_treino': train_r2_region,
    'r2_teste': test_r2_region,
    'mae_treino': train_mae_region,
    'mae_teste': test_mae_region,
    'equacao': f"% Voto Lula = {model_region.intercept_:.2f} + ({model_region.coef_[0]:.2f} × % Bolsa Família) + ({model_region.coef_[1]:.2f} × Nordeste)",
}

metrics_path = os.path.join("..", "data", "metrics", "regional_model_metrics.pkl")
with open(metrics_path, "wb") as f:
    pickle.dump(regional_metrics, f)
print(f"✅ Métricas do modelo regional salvas em: {metrics_path}")

# Save the trained regional model itself
model_path = os.path.join("..", "data", "models", "regional_model_bolsa_familia.pkl")
with open(model_path, "wb") as f:
    pickle.dump(model_region, f)
print(f"✅ Modelo regional salvo em: {model_path}")

In [ ]:
# Função auxiliar para carregar métricas nos outros notebooks
def load_regional_model_metrics():
    """
    Função para carregar as métricas do modelo regional (Bolsa Família + Nordeste) nos outros notebooks.
    
    Returns:
        dict: Dicionário com todas as métricas do modelo regional
    """
    
    metrics_file = os.path.join("..", "data", "metrics", "regional_model_metrics.pkl")
    
    if not os.path.exists(metrics_file):
        raise FileNotFoundError(f"Arquivo de métricas não encontrado: {metrics_file}")
    
    with open(metrics_file, 'rb') as f:
        metrics = pickle.load(f)
    
    return metrics

# Salvar a função em um arquivo Python para importação
utils_dir = os.path.join("..", "utils")
os.makedirs(utils_dir, exist_ok=True)

# Criar arquivo utils/model_utils.py
utils_content = '''"""
Utilitários para carregar métricas dos modelos de regressão
"""

def load_modelo_inicial_metrics():
    """
    Função para carregar as métricas do modelo inicial nos outros notebooks.
    
    Returns:
        dict: Dicionário com todas as métricas do modelo inicial
    """
    metrics_file = os.path.join("..", "data", "metrics", "modelo_inicial_metrics.pkl")
    
    if not os.path.exists(metrics_file):
        raise FileNotFoundError(f"Arquivo de métricas não encontrado: {metrics_file}")
    
    with open(metrics_file, 'rb') as f:
        metrics = pickle.load(f)
    
    return metrics

def load_regional_model_metrics():
    """
    Função para carregar as métricas do modelo regional (Bolsa Família + Nordeste) nos outros notebooks.
    
    Returns:
        dict: Dicionário com todas as métricas do modelo regional
    """
    metrics_file = os.path.join("..", "data", "metrics", "regional_model_metrics.pkl")
    
    if not os.path.exists(metrics_file):
        raise FileNotFoundError(f"Arquivo de métricas não encontrado: {metrics_file}")
    
    with open(metrics_file, 'rb') as f:
        metrics = pickle.load(f)
    
    return metrics
'''

utils_file = os.path.join(utils_dir, "model_utils.py")
with open(utils_file, 'w', encoding='utf-8') as f:
    f.write(utils_content)

print("=== FUNÇÕES EXPORTADAS PARA IMPORTAÇÃO ===")
print(f"📁 Arquivo criado: {utils_file}")
print(f"📦 Agora você pode importar nos outros notebooks:")
print(f"   from utils.model_utils import load_modelo_inicial_metrics, load_regional_model_metrics")

# Exemplo de como usar nos outros notebooks
print("\n=== COMO USAR NOS OUTROS NOTEBOOKS ===")
print("""
# No início dos outros notebooks, adicione:
sys.path.append('..')

# Usar as métricas
regional_metrics = load_regional_model_metrics()
print(f"Modelo regional R²: {regional_metrics['r2_teste']:.4f}")
print(f"Modelo regional equação: {regional_metrics['equacao']}")
""")

# Testar a função
print("\n=== TESTE DA FUNÇÃO ===")
try:
    test_metrics = load_regional_model_metrics()
    print(f"✅ Função funciona! R² = {test_metrics['r2_teste']:.4f}")
except Exception as e:
    print(f"❌ Erro ao testar função: {e}")

print(f"\n🎯 PRONTO PARA OS PRÓXIMOS NOTEBOOKS!")
print(f"   → As métricas estão salvas em: regional_model_metrics.pkl")
print(f"   → Use: from utils.model_utils import load_regional_model_metrics")
print(f"   → Muito mais limpo que redefinir a função toda vez!")
def load_modelo_inicial_metrics():
    """
    Função para carregar as métricas do modelo inicial nos outros notebooks.
    
    Returns:
        dict: Dicionário com todas as métricas do modelo inicial
    """